In [2]:
from metafid.deriv import Pricing
from metafid.deriv import OptionStrategy
import pandas as pd
import jdatetime

In [3]:
today = jdatetime.date.today().strftime("%Y-%m-%d")
today

'1401-12-19'

In [4]:
call = pd.read_excel("call.xlsx")
put = pd.read_excel("put.xlsx")

In [5]:
bsm = Pricing()
o_strategy = OptionStrategy()

In [6]:
bs_call = bsm.bs(call)
bs_call.bs = bs_call.bs.round(1)

In [7]:
columns = ["j_date", "ticker","o_ticker", "t", "bs", "strike_price"] + [i for i in bs_call.columns if "adj" in i]
df_bs_call = bs_call[columns][bs_call.o_adj_final > 0]

In [8]:
bs_put = bsm.bs(put)

In [9]:
df_bs_put = bs_put[columns][bs_put.o_adj_final > 0]
df_bs_put.bs = df_bs_put.bs.round(1)
df_bs_put.tail()

,j_date,ticker,o_ticker,t,bs,strike_price,adj_open,adj_high,adj_low,adj_close,adj_final,o_adj_open,o_adj_high,o_adj_low,o_adj_close,o_adj_final
126820,1401-11-30,خگستر,طستر2010,84,1338.3,5500.0,3745,3884,3735,3773,3795,850.0,850.0,850.0,850.0,850.0
126864,1401-12-06,خگستر,طستر2010,78,919.8,5500.0,4140,4308,4140,4308,4272,999.0,999.0,999.0,999.0,999.0
126900,1401-12-10,خگستر,طستر2008,74,369.6,4500.0,4289,4289,3993,3993,4077,500.0,500.0,500.0,500.0,500.0
126933,1401-12-15,خگستر,طستر2008,69,296.8,4500.0,4198,4300,4124,4184,4209,500.0,500.0,500.0,500.0,500.0
131286,1401-12-10,شبندر,طبدر2042,60,1683.7,13000.0,10680,10680,10010,10150,10340,2500.0,2500.0,2500.0,2500.0,2500.0


### `covered call strategy`

In [11]:
df_covered_call = o_strategy.covered_call(df_bs_call)
df_covered_call.to_excel("rep_covered_call.xlsx")

### `married put strategy`

In [12]:
df_married_put = o_strategy.married_put(df_bs_put)
df_married_put.to_excel("rep_married_put.xlsx")

### `bull call spread strategy`

In [14]:
bull_call_spread = o_strategy.bull_call_spread(bs_call[bs_call.j_date=="1401-12-16"])
bull_call_spread.to_excel("rep_bull_call_spread.xlsx")
bull_call_spread.head()

,o_ticker,strike_price,t,bs,adj_final,o_adj_final,max_ptnl_loss,max_ptnl_profit,current_profit
0,"(ضهرم1211, ضهرم1210)","(18000.0, 16000.0)",6,"(344.9, 2335.6)",18270,"(1180.0, 2074.0)",-894.0,1106.0,1106.0
1,"(ضهرم1211, ضهرم1209)","(18000.0, 15000.0)",6,"(344.9, 3331.5)",18270,"(1180.0, 3092.0)",-1912.0,1088.0,1088.0
2,"(ضهرم1211, ضهرم1208)","(18000.0, 14000.0)",6,"(344.9, 4327.4)",18270,"(1180.0, 3576.0)",-2396.0,1604.0,1604.0
3,"(ضهرم1211, ضهرم1207)","(18000.0, 13000.0)",6,"(344.9, 5323.3)",18270,"(1180.0, 4739.0)",-3559.0,1441.0,1441.0
4,"(ضهرم1211, ضهرم1206)","(18000.0, 12000.0)",6,"(344.9, 6319.2)",18270,"(1180.0, 5512.0)",-4332.0,1668.0,1668.0


In [ ]:
"https://optionstrategiesinsider.com/blog/28-option-strategies-that-all-options-traders-should-know/"

### `bear call spread strategy`

In [8]:
bear_call_spread = o_strategy.bear_call_spread(bs_call[bs_call.j_date=="1401-12-16"])
bear_call_spread.to_excel("rep_bear_call_spread.xlsx")
bear_call_spread.head()

,o_ticker,strike_price,t,bs,adj_final,o_adj_final,max_ptnl_loss,max_ptnl_profit,current_profit
0,"(ضهرم1200, ضهرم1201)","(7000.0, 7500.0)",6,"(11298.7, 10800.8)",18270,"(10374.0, 9314.0)",560.0,1060.0,560.0
1,"(ضهرم1200, ضهرم1202)","(7000.0, 8000.0)",6,"(11298.7, 10302.8)",18270,"(10374.0, 8974.0)",400.0,1400.0,400.0
2,"(ضهرم1200, ضهرم1203)","(7000.0, 9000.0)",6,"(11298.7, 9306.9)",18270,"(10374.0, 8249.0)",125.0,2125.0,125.0
3,"(ضهرم1200, ضهرم1204)","(7000.0, 10000.0)",6,"(11298.7, 8311.0)",18270,"(10374.0, 7875.0)",-501.0,2499.0,-501.0
4,"(ضهرم1200, ضهرم1205)","(7000.0, 11000.0)",6,"(11298.7, 7315.1)",18270,"(10374.0, 6689.0)",-315.0,3685.0,-315.0


In [10]:
from collections import namedtuple
from itertools import combinations
from operator import add

In [28]:
def call_back_spread(df, times):
    stg = namedtuple("Strategy", "buy sell")
    groups = df.groupby(by=["ticker", "t"])
    df_ = pd.DataFrame()
    for name, group in groups:
        group.reset_index(inplace=True)
        group = group.sort_values(by=["strike_price"], ascending=False)
        if len(group) > 1:
            combo_o_ticker = list(stg(buy=b, sell=s) for b, s in combinations(group.o_ticker, 2))
            combo_strike_price = list(stg(buy=b, sell=s) for b, s in combinations(group.strike_price, 2))
            combo_o_adj_final = list(stg(buy=b, sell=s) for b, s in combinations(group.o_adj_final, 2))
            combo_bs = list(stg(buy=b, sell=s) for b, s in combinations(group.bs, 2))
            premium = [-pb*times + ps for pb, ps in combo_o_adj_final]
            max_pot_loss = list(map(add, [-sb + ss for sb, ss in combo_strike_price], premium))
            current_profit = []
            for i in range(len(combo_strike_price)):
                if all(s >= group.adj_final.values[0] for s in combo_strike_price[i]):
                    current_profit.append(premium[i])
                elif combo_strike_price[i].buy >=group.adj_final.values[0]:
                    current_profit.append(- group.adj_final.values[0] + combo_strike_price[i].sell + premium[i])
                else:
                    current_profit.append((- group.adj_final.values[0] + combo_strike_price[i].sell)+(group.adj_final.values[0] - combo_strike_price[i].buy)*times + premium[i])

            df_group = pd.DataFrame({
                "o_ticker": combo_o_ticker,
                "strike_price": combo_strike_price,
                "t": group.t.values[0],
                "bs": combo_bs,
                "adj_final": group.adj_final.values[0],
                "o_adj_final": combo_o_adj_final,
                "max_pot_loss": max_pot_loss,
                "current_profit": current_profit,
            })
            df_ = pd.concat([df_, df_group])
    return df_

In [32]:
call_back_spread(df_bs_call[df_bs_call.j_date=="1401-12-15"], 2).to_excel("rep_call_back_spread.xlsx")


In [31]:
16000.0 -  18000.0 + (- 594.0*2 +1140.0 )

-2048.0

In [23]:
4083.0 - 3341.0

742.0